<center>

<img src="../img/skyline.jpg" alt="Descripción de la imagen" width="900" height="300">

</center>



<center>

# Predicción de precios en el mercado inmobiliario de Sevilla mediante Machine Learning

</center>

## Introducción

En este proyecto, vamos a intentar predecir mediante un modelo de machine learning, el precio de alquiler o de venta de una vivienda, en el area metropolitana de Sevilla, según las características de la vivienda.

Para ello, disponemos de 2 bases de datos, obtenidas de la [API de Idealista](https://developers.idealista.com/access-request). Las bases de datos extraídas de dicha API son:
* Alquiler: una base de datos formato json, de la cual hemos obtenido una dataframe de 1001 registros y 41 variables.
* Venta: una base de datos formato json, de la cual hemos obtenido una dataframe de 1700 registro y 41 variables.

El proceso que hemos seguido para obtener nuestro modelo de predicción de machine learning es:

1. Extracción de datos.

2. Estudio y tratamiento de dataframes:
               
3. Entrenamiento y evaluación de nuestro modelo:
            
4. Predicciones.

## Extracción de datos

Podrás accede al notebook donde se extraen los datos:

- [Fuentes](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/notebooks/01_Fuentes.ipynb)

La extracción de los datos se ha realizado de la API de idealista, la cual nos proporcionó unas keys y un número de llamadas al mes limitado (100 llamadas).

La extracción de datos consta de 3 partes:

1. Token de acceso: mediante una funcion donde los atributos son las 2 keys que se me proporcionan, obtenemos una clave cuya variable llamaremos access_token la cual, nos dará autorización para acceder a la API.
-------------------------------------------------------

2. Acceso a la API: aquí creamos la función con la cual, haremos las llamadas a la API. Los atributos serán los parámetros con los que realizaremos nuestra búsqueda:
    * api_key, api_secret: las keys proporcionadas.
    * country: pais donde buscaremos (es)
    * operation: alquiler o ventas (rent o sale)
    * property_type: tipo de propiedad que buscamos (home)
    * center: coordenadas a partir de las cuales, iniciaremos nuestra búsqueda
    * distance: radio en metros con el que ampliaremos nuestra búsqueda
    * locale: idoma de búsqueda ('es')
    * max_items: número de registros por cada llamada. Introducimos 50 ya que es el máximo permitido
    * num_page: número de página según el total de registros obtenidos con nuestra búsqueda
    * order: orden de búsqueda
    * sort: ordenació según el orden de búsqueda ('asc' o 'desc')
    
---------------------------------------------------------------

3. Llamadas a la API: 
    * Mediante un bucle for, vamos realizando llamadas hasta ibtener todos nuestros datos. Ya que el máximo de registros por llamada son 50 y los registros que tenemos extraídos han sido 1001 para alquiler y 1700 para venta, nos vamos creando una lista con los números de página por los que vamos a iterar.
    * Mediante un bucle for, voy cogiendo de los archivos json extraídos, los apartados que necesito para crear con ellos las dataframes, las cuales, las iré metiendo en un diccionario llamado dataframes.
    * Mediante un bucle for, voy recorriendo ese diccionario para ir concatenando cada una de esas dataframes a la dataframe final llamada df_total.




## Estudio y tratamiento de dataframes

Podrás accede al notebook donde realizamos el tratamiento de dataframe:

- [LimpiezaEDA](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/notebooks/02_LimpiezaEDA.ipynb)

- [LimpiezaEDA_venta](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/notebooks/06_LimpiezaEDA_venta.ipynb)

Los pasos que hemos seguidos para el estudio y tratamiento de nuestras dataframe han sido:

1. Limpieza de datos:

    * Eliminación de columnas irrelevantes: se eliminan las columnas que no aportan nada a nuestro estudio.
    * Eliminación de registros anómalos: tras examinar nuestros datos, vemos que hay registros con algunas anomalías, los cuales, pueden entorpecer el futuro entrenamiento de nuestro modelo. Estos registros son:
        - Alquiler de habitaciones.
        - Alquileres por días.
        - Viviendas donde en el tamaño han puesto el de la parcela. 
    * Resetear índices: siempre que eliminamos registros, es conveninete resetear los índices para no tener problemas al recorrer la dataframe con bucle for.

-------------------------------------------------------------------------

2. Nuevas features: creamos algunas features que consideramos que serán importantes para la predicción de nuestro modelo.

    * Parking: creamos una columna binaria en la que 1 es que el parking está incluido en el precio y 0 que no lo está.
    * Población-distrito: en este caso, dividimos los terminos municipales a su vez, en municipio y distrito. Para ello hemos creado 3 columnas diferentes:
        - codigo_distrito: será asignado en orden de menor a mayor, tras calcular la media de la columna priceByArea de la agrupación población-distrito
        - poblacion / distrito: que será la concatenación del municipio y su distrito
        - precio_area / distrito: precioByArea medio por municipio/distrito
    * Codigo_tipo: codificamos el tipo de vivienda de menor a mayor según la media de precio de su agrupación
    * Psicina: asignamos 1 a las viviendas que tienen piscina y 0 a las que no la tienen
    * Muebles: asignamos 2 a las viviendas amuebladas y 0 a las sin muebles y el 1 a las que están sin determinar.

-----------------------------------------------------------------------------

3. df_num: creamos una dataframe solo con valores numéricos y a partir de la cual, analizaremos las features que vamos a utilizar para nuestro entrenamiento.
    * Total_rooms: para evitar la multicolinealidad, creamos una columna con la suma de habitaciones y baños

----------------------------------------------------------------------------

4. Análisis: Tras analizar las correlaciones y la importancia que puede tener cada feature en el entrenado del modelo, la df_num constará de las siguientes features:
    * size
    * codigo_distrito
    * parking
    * codigo_tipo
    * piscina
    * total_rooms



## Entrenamiento y evaluación de nuestro modelo:

Podrás accede al notebook donde realizamos el entrenamiento y la evaluación del modelo:

- [Entrenamiento_Evaluación](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/notebooks/03_Entrenamiento_Evaluacion.ipynb)

- [Entrenamiento_Evaluación_venta](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/notebooks/07_Entrenamiento_Evaluacion_venta.ipynb)

Para la elección del modelo, primero hemos realizado una búsqueda mediante un Pipeline y utilizando la función GridsearchCV, eligiendo 5 modelos y algunos parametros y obteniendo como resultado mas exitoso, el modelo Gradient Boosting Regressor.
A pesar de que ya sabemos que nuestra elección será el modelo GradientBoostingRegressor, hemos hecho una busqueda individual de los mejores parametros en cada modelo, evaluándolos a su vez, y obteniendo los siguientes resultados (podrás acceder a los parametros de cada modelo en los siguientes enlaces):

| Alquiler | Venta |
| --- | --- |
| [GradientBoostingRegressor](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_GBR.yaml) | [GradientBoostingRegressor](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_GBR_venta.yaml) |
|   - MAE: 164.672   |   - MAE: 52323.585   |
|   - MAPE: 0.149   |   - MAPE: 0.217   |
|   - RMSE: 253.197   |   - RMSE: 112136.034   |
| [DecisionTreeRegressor](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_DTR.yaml) | [DecisionTreeRegressor](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_DTR_venta.yaml) |
|   - MAE: 225.241   |   - MAE: 72566.664   |
|   - MAPE: 0.200   |   - MAPE: 0.285   |
|   - RMSE: 335.238   |   - RMSE: 171864.575   |
| [SVR](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_SVR.yaml) | [SVR](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_SVR_venta.yaml) |
|   - MAE: 190.993   |   - MAE: 82393.539   |
|   - MAPE: 0.165   |   - MAPE: 0.316   |
|   - RMSE: 299.148   |   - RMSE: 196879.176   |
| [KNeighborsRegressor](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_KNR.yaml) | [KNeighborsRegressor](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_KNR_venta.yaml) |
|   - MAE: 191.520   |   - MAE: 61442.863   |
|   - MAPE: 0.171   |   - MAPE: 0.237   |
|   - RMSE: 282.094   |   - RMSE: 136764.593   |
| [RandomForestRegressor](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_RFR.yaml) | [RandomForestRegressor](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/models/modelo_RFR_venta.yaml) |
|   - MAE: 203.004   |   - MAE: 71844.231   |
|   - MAPE: 0.192   |   - MAPE: 0.295   |
|   - RMSE: 303.863   |   - RMSE: 162976.846   |




## Predicciones

Para predecir el alquiler o venta de una vivienda mediante la introducción de nuevos input, hemos realizado esta [aplicación de streamlit](https://github.com/ManuelRF86/proyecto_ML_alquiler/blob/main/app/app.py), donde introduciendo las características de una vivienda, la aplicación te calculará el precio recomendado para esa vivienda.
Para ejecutarla, solo tendras que, desde el terminal, ubicarte en la carpeta app de nuestro repositorio y ejecutar el comando: streamlit run app.py